In [15]:
import fileinput

data = [line.strip() for line in fileinput.input("input.txt")]

point_data = []
for line in data:
    x, y = line.split(",")
    x = int(x)
    y = int(y.strip())
    point_data.append([x, y])

In [16]:
def construct_bounding_rectangle(all_points):
    '''
    Theory: Once a point has expanded to the bounding rectangle, it will continue to expand infinitely.
    '''
    min_x = 10000000
    min_y = 10000000
    max_x = 0
    max_y = 0
    
    for point in all_points:
        x, y = point.split(",")
        x = int(x)
        y = int(y.strip())
        
        min_x = min(min_x, x)
        min_y = min(min_y, y)
        max_x = max(max_x, x)
        max_y = max(max_y, y)
    
    return min_x, min_y, max_x, max_y
min_x, min_y, max_x, max_y = construct_bounding_rectangle(data)

In [17]:
from collections import Counter, defaultdict

board = defaultdict(str)
labeled_data = [(str(i), point_data[i]) for i in range(len(point_data))]

In [18]:
# Get the coordinates that are on the bounding rectangle.
bounding_set = set()
for label, point in labeled_data:
    if point[0] == min_x or point[0] == max_x or point[1] == min_x or point[1] == max_y:
        bounding_set.add(label)

In [23]:
# Strategy: Do BFS expansion of each node. Stop once there are 4 nodes left to expand.
# Mark each area with a tuple (Label, iteration)
# Replace with '.' if the iteration number is the same.

move_list = []
for elem in labeled_data:
    move_list.append([elem[1]])
    
board = defaultdict()

iteration = 0
number_left = len(move_list)
counter = Counter() # Counts the number of elements per label
touched_bounding_box = set()
board = {}

while (number_left != 0):
    
    # Iterate over each element.
    for i in range(len(move_list)):
        started = False
        num_expanded = 0
        
        current_tips = move_list[i].copy()
        move_list[i] = [] # Burn the list.
        for tip in current_tips:
            started = True
            x = tip[0]
            y = tip[1]

            value_here = board.get((x,y))
            if not value_here or value_here[0] != '.':
                # No need to continue if someone replaced us with a '.'
                left = [x-1, y]
                right = [x+1, y]
                up = [x, y+1]
                down = [x, y-1]
                
                for new_point in [left, right, up, down]:
                    new_tuple = (new_point[0], new_point[1])
                    val = board.get(new_tuple)
                    
                    if val is None:
                        # No one has claimed this yet, so we will tentatively claim it.
                        # Optimistically add the new point to the move list.
                        board[new_tuple] = (str(i), iteration)
                        
                        # Check if we're expanding beyond the bounding box.
                        if new_tuple[0] > min_x and new_tuple[0] < max_x and new_tuple[1] > min_y and new_tuple[1] < max_y:
                            move_list[i].append(new_point)
                            counter[str(i)] += 1
                            expanded = True
                            num_expanded += 1
                        else:
                            # Populate the blacklist of box touchers.
                            touched_bounding_box.add(str(i))
                    
                    elif val[1] == iteration and val[0] != str(i):
                        # This value was just placed here, so make it '.'
                        board[new_tuple] = ('.', iteration)
                        counter[val[0]] -= 1
                        
            
        if started and num_expanded == 0:
            # We didn't expand any nodes, so this element is dead.
            # We aren't being exact with our iterations, so it may actually die one round later.
            number_left -= 1
                        
    # We've processed the whole list, so bump up the iteration.
    iteration += 1

In [22]:
candidates = []
for elem in counter:
    if elem not in touched_bounding_box:
        candidates.append(counter[elem])
        
print(max(candidates))

4284
